<a href="https://colab.research.google.com/github/csabiu/ML_tutorial/blob/master/sdss_2pcf_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook will guide you through a realistic analysis of galaxy survey data.
We will download real data and then measure its 2-point correlation function.
We will make a theoretical prediction of the 2-point correlaiton function and compare with that of the observation.

In [31]:
import scipy
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
plt.rcParams.update({'font.size': 15})
plt.rcParams.update({'figure.figsize': [10.0, 10.0]})

Install CAMB for theory prediction \\
Install KSTAT for making measurement

In [ ]:
!pip install camb
!git clone https://github.com/csabiu/kstat.git

Make KSTAT manually

In [3]:
output = os.system("cd kstat/src; make ; cd ../../")

### Download SDSS eBOSS DR12 CMASS data and random catalogs
search in https://data.sdss.org/sas/ for other catalogues if you want to try a different target eg, ELG, QSO - or even newer eBOSS data!

In [ ]:
!wget https://data.sdss.org/sas/dr12/boss/lss/galaxy_DR12v5_CMASS_South.fits.gz
!wget https://data.sdss.org/sas/dr12/boss/lss/random0_DR12v5_CMASS_South.fits.gz

### Extract relavent columns from the fits file
Angular positions (RA, dec), redshift (z) and weights

In [6]:
hdul = fits.open('galaxy_DR12v5_CMASS_South.fits.gz')
ra=hdul[1].data['RA']
dec=hdul[1].data['DEC']
z=hdul[1].data['Z']
wtot=hdul[1].data['WEIGHT_SYSTOT']
wcp=hdul[1].data['WEIGHT_CP']
wnoz=hdul[1].data['WEIGHT_NOZ']
wfkp=hdul[1].data['WEIGHT_FKP']
wgal=wtot*wfkp*(wcp+wnoz-1.0) #Eq 2 from https://arxiv.org/pdf/1607.03154.pdf


### Plot the redshift distribution

In [ ]:
ind=np.where((z>0.4) & (z<0.7)) #cut the redshift range
plt.hist(z[ind],100)
plt.xlabel('z',fontsize=20)
plt.ylabel('dN',fontsize=20)
print("mean redshift of the galaxies :",np.mean(z))

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(np.mod(ra+50,360),dec,',')
plt.xlabel('RA',fontsize=20)
plt.ylabel('dec',fontsize=20)

## Use astropy to convert from angle and redshift to cartesean comoving x,y,z positions

In [9]:
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u
from astropy.coordinates import SkyCoord

ind=np.where((z>0.4) & (z<0.7))
cosmo=FlatLambdaCDM(Om0=0.30,H0=100) # sets the cosmological model working in units Mpc/h with Ho=100
dist=cosmo.comoving_distance(z[ind]).value  # convert redshift to distance

c = SkyCoord(ra=ra[ind]*u.degree, dec=dec[ind]*u.degree, distance=dist*u.Mpc) # convert to x,y,z
data=np.transpose((c.cartesian.x,c.cartesian.y,c.cartesian.z, wgal[ind]))
np.savetxt("gal.txt",data) # save file as 4 columns x,y,z,w 

### Load the Random Catalogue and repeat the same proceedure 

In [11]:
hdul = fits.open('random0_DR12v5_CMASS_South.fits.gz')
ra=hdul[1].data['RA']
dec=hdul[1].data['DEC']
z=hdul[1].data['Z']
wrand=hdul[1].data['WEIGHT_FKP']

In [ ]:
ind=np.where((z>0.4) & (z<0.7))
plt.hist(z[ind],100)
plt.xlabel('z',fontsize=20)
plt.ylabel('dN',fontsize=20)
print("mean redshift of the randoms :",np.mean(z))

Plotting the random point angular positions we see that they look almostly exactly like the galaxies as they should

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(np.mod(ra[::50]+50,360),dec[::50],',') # plot a random sample 1/50
plt.xlabel('RA',fontsize=20)
plt.ylabel('dec',fontsize=20)

In [14]:
dist=cosmo.comoving_distance(z[ind]).value

c = SkyCoord(ra=ra[ind]*u.degree, dec=dec[ind]*u.degree, distance=dist*u.Mpc)
data=np.transpose((c.cartesian.x,c.cartesian.y,c.cartesian.z, wrand[ind]))
np.savetxt("ran.txt",data[::50,:]) # save a subsample of the data (1/50)

## Run my KSTAT code (~1 min)
20 measurement bins logarithmically spaced between 1 and 60 Mpc

In [ ]:
!kstat/bin/2pcf -gal gal.txt -ran ran.txt -rmin 1.0 -rmax 60.0 -nbins 20 -log .true. -wgt .true.

Load and plot the result file

In [16]:
!head -1 result.txt # the columns of the result file 

R_min, R_max, DD, DR, RR, XI_natural, XI_Davis, XI_LS (most accurate)


In [ ]:
x=np.loadtxt('result.txt',skiprows=1)

xi_natural=x[:,5]
xi_davis=x[:,6]
xi_LS=x[:,7]
s=np.mean(x[:,0:2],axis=1)

plt.figure(figsize=(10,8))
plt.loglog(s,xi_natural,'rs',markerfacecolor='none', ms=10,label=r'$\xi_{natural}$')
plt.loglog(s,xi_davis,'bo',markerfacecolor='none', ms=10,label=r'$\xi_{davis}$')
plt.loglog(s,xi_LS,'k.',label=r'$\xi_{LS}$')

plt.legend()
plt.xlabel('r [Mpc]',fontsize=20)
plt.ylabel(r'$\xi$',fontsize=20)

### Lets start to make the theory prediction with CAMB

In [19]:
import camb
from camb import model, initialpower

### This is how you set up an instance of CAMB for the matter power spectrum with a particular cosmology 

In [20]:
kmax=100          # maximum k range
k_per_logint = 10  # number of points per log interval
zs = [0.55]        # redshift (mean of the galaxy sample) 

pars = camb.CAMBparams()
pars.set_cosmology(H0=68.0, ombh2=0.022, omch2=0.12) 
pars.InitPower.set_params(ns=0.965,As=2.0905e-09)
pars.WantTransfer = True
pars.set_matter_power(redshifts=zs, kmax=kmax, k_per_logint=k_per_logint, silent=True)
results = camb.get_results(pars)


PKint = results.get_matter_power_interpolator(nonlinear=True, 
    hubble_units=True, k_hunit=True)

### Lets plot the power spectrum

In [ ]:
kh_nonlin=np.exp(np.arange(-5.0,3.8,0.0001)) # set k steps
pk_nonlin=PKint.P(0.55, kh_nonlin)           # get P at those k and reshift z of sample
plt.loglog(kh_nonlin,pk_nonlin,'-')

plt.xlabel('k [1/Mpc]',fontsize=20)
plt.ylabel(r'$P(k)$',fontsize=20)

Need to numerically integrate $$ \xi(r)=\frac{1}{2\pi^2}\int k^2P(k) j_0 dk$$

use trapz function to numerically integrate over the integrand which is just product of 1-D arrays - for smooth integrands with enough samples trapz is usually accurate enough

In [22]:
bias=2.2 #galaxy bias parameter

xi=[]
rr=np.arange(1,60,2)  # set range in r scale Mpc/h
for r in rr:
    xi.append(bias**2*np.trapz(kh_nonlin**2*pk_nonlin*np.sin(kh_nonlin*r)/(kh_nonlin*r),kh_nonlin)/(2*np.pi**2) )

In [ ]:
plt.loglog(rr,xi)

plt.xlabel('r [Mpc]')
plt.ylabel(r'$\xi$')

### Plot the theory and data together

In [ ]:
plt.loglog(rr,xi,'-',label='theory')
plt.loglog(s,xi_LS,'.',label='SDSS data')

plt.legend()
plt.xlabel('r [Mpc]')
plt.ylabel(r'$\xi$')

### Run KSTAT with the anisotropic options (~2 mins)

In [27]:
!kstat/bin/2pcf -gal gal.txt -ran ran.txt -rmin 1.0 -rmax 60.0 -nbins 20 -wgt .true. -iso ANISO -decp SIGPI > out.txt

STOP Ciao for now!


## Load the result file and rearrange and plot
We can clearly see the squashing effect in the radial direction


In [ ]:
x=np.loadtxt("result.txt",skiprows=1)
sig=np.mean(x[:,0:2],axis=1).reshape((20,20)) 
pi=np.mean(x[:,2:4],axis=1).reshape((20,20))
xi=x[:,9].reshape((20,20)).transpose()

plt.figure(figsize=(10,10))

plt.contour(pi,sig,xi,levels=[0.04,0.1,0.2,0.7],colors=['r','g','b','k'])
plt.xlim((2,55))
plt.ylim((2,55))

#########################################################################
#### plotting circular contours to show the squashing effect in the data
circ1=plt.Circle((0,0),12,color='k', fill=False,linestyle='--')
circ2=plt.Circle((0,0),26,color='b', fill=False,linestyle='--')
circ3=plt.Circle((0,0),35,color='g', fill=False,linestyle='--')
circ4=plt.Circle((0,0),50,color='r', fill=False,linestyle='--')
ax = plt.gca()
ax.add_patch(circ1)
ax.add_patch(circ2)
ax.add_patch(circ3)
ax.add_patch(circ4)
#########################################################################
plt.xlabel(r'$\sigma$ [Mpc/h]')
plt.ylabel(r'$\pi$ [Mpc/h]')

### Anisotropic Analysis
Use the kaiser formula \\
$$ P^s_g(k,\mu)= P^r_m(k)( b_g +f\mu^2)^2$$

In [40]:
ff=1.1   # growth
bias=1.5 # galaxy bias

pk_ani=np.zeros((60,60))

for i in range(1,60):    #1->60 Mpc sigma-pi
  for j in range(1,60):
    rr=np.sqrt(i**2+j**2)
    mu=j/rr
    pk=pk_nonlin*(bias+ff*mu**2)**2
    pk_ani[i,j]=np.trapz(kh_nonlin**2*pk*np.sin(kh_nonlin*rr)/(kh_nonlin*rr),kh_nonlin)/(2*np.pi**2)

### Plot the theory and data together
Its a pretty good match!

In [ ]:
plt.figure(figsize=(10,10))
plt.contour(pi,sig,(xi),levels=[0.04, 0.1,0.3,1], colors='k')
plt.contour(range(1,61),range(1,61),(pk_ani),levels=[0.04, 0.1,0.3,1], colors='r')
plt.xlim((2,55))
plt.ylim((2,55))
plt.xlabel(r'$\sigma$ [Mpc/h]',fontsize=20)
plt.ylabel(r'$\pi$ [Mpc/h]',fontsize=20)

Adding the Alcock Paczynski Effect \\
$$\xi(\sigma,\pi) = \xi(\alpha_{\perp}\sigma_{fid},\alpha_{\parallel}\pi_{fid})$$ \\
$$\alpha_{\perp}=\frac{D_A(z)r_{d,\rm fid}}{D^{\rm fid}_A(z)r_{d}}, ~ \alpha_{\parallel}=\frac{H^{\rm fid}(z)r_{d,\rm fid}}{H(z)r_{d}}$$

In [41]:
ff=0.7       # growth
bias=1.8     # galaxy bias
alphapi=1.05
alphasigma=1.00

pk_ani2=np.zeros((60,60))

for i in range(1,60):    #1->60 Mpc sigma-pi
  for j in range(1,60):
    rr=np.sqrt((i*alphasigma)**2+(j*alphapi)**2)
    mu=(j*alphapi)/rr
    pk=pk_nonlin*(bias+ff*mu**2)**2
    pk_ani2[i,j]=np.trapz(kh_nonlin**2*pk*np.sin(kh_nonlin*rr)/(kh_nonlin*rr),kh_nonlin)/(2*np.pi**2)

In [ ]:
plt.figure(figsize=(10,10))
#plt.contour(pi,sig,(xi),levels=[0.04, 0.1,0.3,1], colors='k')
plt.contour(range(1,61),range(1,61),(pk_ani),levels=[0.04, 0.1,0.3,1], colors='r')
plt.contour(range(1,61),range(1,61),(pk_ani2),levels=[0.04, 0.1,0.3,1], colors='b')
plt.plot([-1,-1],[-1,-1],'r-',label='fiducial model')
plt.plot([-1,-1],[-1,-1],'b-',label=r'$\alpha_{\parallel}=1.05$')
plt.xlim((2,55))
plt.ylim((2,55))
plt.legend()
plt.xlabel(r'$\sigma$ [Mpc/h]')
plt.ylabel(r'$\pi$ [Mpc/h]')

## Large Scale and BAO
But lets make JK samples to compute errorbars

In [ ]:
!kstat/bin/jk.sh gal.txt ran.txt 5 3

Run code (~8mins)

In [ ]:
!kstat/bin/2pcf -gal gal.txt.jk -ran ran.txt.jk -rmin 50.0 -rmax 125.0 -nbins 15 -wgt .true. -err 'read' -nerr 125

The isotropic BAO position is parameterised by the scale dilation
parameter $$\alpha=\frac{D_v(z)r_{d,\rm fid}}{D^{\rm fid}_v(z)r_{d}}$$

In [150]:
bias=2.4 #galaxy bias parameter

xi=[]
rr=np.arange(50,130,2)
alpha=1.06
for r in rr:
    xi.append(bias**2*np.trapz(kh_nonlin**2*pk_nonlin*np.sin(kh_nonlin*alpha*r)/(kh_nonlin*alpha*r),kh_nonlin)/(2*np.pi**2) + 0.002)

In [ ]:
x=np.loadtxt('result.txt',skiprows=1)
r=np.mean(x[:,0:2],axis=1)
plt.errorbar(r,x[:,2]*r**2,yerr=x[:,3]*r**2,ls='none')
plt.plot(r,x[:,2]*r**2,'ks')
plt.plot(rr,xi*rr**2,'-')
plt.xlabel('r [Mpc/h]')
plt.ylabel(r'$\xi$')


Fit the BAO shift parameter
$$\chi^2=\sum\frac{(\xi_{obs}-\xi_{theory})^2}{\sigma^2_{\xi}}$$

In [ ]:
from scipy.optimize import minimize
ckms=299792.458 #km/s

x=np.loadtxt('result.txt',skiprows=1)
dists=np.mean(x[3:,0:2],axis=1)
xi=x[3:,2]
xi_err=x[3:,3]

def BAOchi2(params):
  alpha=params[0]
  bias=params[1]
  xim=[]
  for r in dists:
    xim.append(bias**2*np.trapz(kh_nonlin**2*pk_nonlin*np.sin(kh_nonlin*alpha*r)/(kh_nonlin*alpha*r),kh_nonlin)/(2*np.pi**2) + 0.002)

  chi2=np.sum((xi-xim)**2 / xi_err**2)
  return chi2


alpha_bestfit=1.0
bias_bestfit=2.0

bestfit=minimize(BAOchi2, [alpha_bestfit, bias_bestfit],method='Nelder-Mead')
print('Best-fit parameters:', bestfit['x'], r'$\chi^2$',bestfit['fun'])
print(bestfit)

#1.07*ckms/cosmo.H(0.55).value/rsf(1/1021.0,0.04,0.3,0,Omega_rB,68.0,-1.0,0.0)

A spherically averaged distance is $$D_v(z)=[cz(1+z)^2D_A(z)^2H(z)^{-1}]^{1/3}$$

$r_d$ is the radius of the sound horizon at the redshift $z_d$​ when baryons got freed from the photon drag. The sound horizon is given by $$r_s(z)=\int^a_0\frac{c_s da'}{a'H(z)}$$ and $$c_s=\frac{c}{3\left(1+\frac{3\Omega_b a}{4\Omega_r}\right)}$$

In [39]:
from astropy.cosmology import FlatwCDM
ckms=299792.458 #km/s

h=0.67
Om=0.30
Ob=0.05
w0=-1.02

cosmo = FlatwCDM(H0=h*100.0, Om0=Om, w0=w0)

############################################
#### load data
# copied data from https://www.sdss.org/science/final-bao-and-rsd-measurements/
DV_data=np.asarray([4.47,18.33])
DV_err=np.asarray([0.17,0.6])
DV_z=np.asarray([0.15,0.85])

DM_data=np.asarray([10.23,13.36,17.86,30.69,37.6,37.3])
DM_err=np.asarray([0.17, 0.21, 0.21, 0.80, 1.9, 1.7])
DM_z=np.asarray([0.38,0.51,0.70,1.48,2.33,2.33])

DH_data=np.asarray([25.00, 22.33, 19.33, 13.26, 8.93, 9.08])
DH_err=np.asarray([ 0.76, 0.58,  0.53, 0.55, 0.28,0.34])
DH_z=np.asarray([0.38,0.51,0.70,1.48,2.33,2.33])
############################################

## sound horizon approx from Eq 12 of arxiv:0907.1660    
rs=153.5*((Ob*h*h/0.02273)**-0.134)*((Om*h*h/0.1326)**-0.255)

H0=ckms/cosmo.H(DH_z).value
DM=cosmo.comoving_distance(DM_z).value



Plot the Comoving distance

In [ ]:
fig,ax = plt.subplots(2,1,sharex=True,gridspec_kw={'height_ratios': [3, 1]})

ax[0].errorbar(DM_z,DM_data,yerr=DM_err,ls='none',marker='.',label='SDSS data')
ax[0].plot(DM_z,DM/rs,'k-',label='Planck best fit')
ax[1].plot(DM_z,DM_z*0,'k--')
ax[1].errorbar(DM_z,DM_data-DM/rs,yerr=DM_err,ls='none',marker='.')
ax[1].set_xlabel('redshift, z')
ax[1].set_ylabel(r'$\Delta D_c$')
ax[0].set_ylabel(r'$D_c$ / $r_{fid}$')
ax[0].legend()
plt.subplots_adjust(wspace=0, hspace=0)

Plot the Hubble distance

In [ ]:
fig,ax = plt.subplots(2,1,sharex=True,gridspec_kw={'height_ratios': [3, 1]},figsize=(10,10))
ax[0].errorbar(DH_z,DH_data,yerr=DH_err,ls='none',marker='.',label='SDSS data')
ax[0].plot(DH_z,H0/rs,'k-',label='Planck best fit')
ax[1].plot(DH_z,DH_z*0,'k--')
ax[0].legend()
ax[1].errorbar(DH_z,DH_data-H0/rs,yerr=DH_err,ls='none',marker='.')
ax[1].set_xlabel('redshift, z')
ax[1].set_ylabel(r'$\Delta D_H$')
ax[0].set_ylabel(r'$D_H$ / $r_{fid}$')
plt.subplots_adjust(wspace=0, hspace=0)

In [5]:
from scipy import integrate 
ckms=299792.458 #km/s

def w(a,w0,wa):
    return w0+(1.0-a)*wa

def eos_integrand(lna,w0,wa):
    a=np.exp(lna)
    wnow=w(a,w0,wa)
    eos_integrand=(1+wnow)
    return eos_integrand

def OmegaDE(a,OmegaDE0,w0,wa):
    #Integrating from today to an earlier time in lna
    factor=integrate.romberg(eos_integrand,0,np.log(a),args=(w0,wa),rtol=1e-3)
    OmegaDE=OmegaDE0*np.exp(-3.0*factor)
    return OmegaDE

def rs_integrand(a,Om_b,Om_m,Om_k,Om_r,H0,w0,wa):
    Om_de0=1-Om_m-Om_k-Om_r
    Om_de=OmegaDE(a,Om_de0,w0,wa)
    # massless species # assume 3 
    Omr_massless=3.0*Om_r*(7.0/8.0)*(4.0/11.0)**(4.0/3.0)
    asquareH=H0*np.sqrt((Om_m*a)+Om_r+Omr_massless+(Om_k*a**2)+(Om_de*a**4))
    Rb=(3.0*Om_b*a)/(4.0*Om_r)
    rs_integrand=ckms/(np.sqrt(3.0*(1.0+Rb))*asquareH)    
    return rs_integrand

def rsf(a,Om_b,Om_m,Om_k,Om_r,H0,w0,wa):
    #In Mpc
    rs=integrate.romberg(rs_integrand,1e-9,a,args=(Om_b,Om_m,Om_k,Om_r,H0,w0,wa),rtol=1e-3)    
    return rs  

#### best fit planck values ####
Omega_mB=0.3153
Omega_rBh2=2.47e-5
Omega_kB=0
H0B=67.36
Omega_rB=Omega_rBh2/(H0B/100.0)**2
Omega_lB=1-Omega_mB-Omega_rB-Omega_kB
Omega_bBh2=0.0224
Omega_bB=Omega_bBh2/(H0B/100.0)**2

#rs=
rsf(1/1021.78,Omega_bB,Omega_mB,0.0000000,Omega_rB,H0B,-1.0,0.0)


150.7973009430952